In [83]:
import pandas as pd 
import datetime
import numpy as np
import matplotlib.pyplot as plt
import re
import os
%matplotlib inline


# Предобработка и соединение файлов с номерами и датами

In [84]:
quart = pd.DataFrame()

for file in os.listdir():
    if '$' not in file and 'xlsx' in file:
        dict_df = pd.read_excel(file, sheet_name=None, dtype=str)
        for key in dict_df.keys():
            name = (file[file.find('.') + 1:file.rfind('.')] + ' ' + key).strip()
            temp_df = dict_df[key][['Дата', 'Клиент: телефон']]
            temp_df['Источник'] = [name] * temp_df.shape[0]
            quart = quart.append(temp_df)

C:\Users\AmanovRA\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [85]:
quart = quart.reset_index(drop=True)


In [86]:
pattern = re.compile(r'([^ ]+)( [^ ]+)')


def string(s):
    pattern = re.compile(r'([^ ]+)( [^ ]+)')
    if pattern.findall(s):
        return pattern.findall(s)[0][0]
    else:
        return s
    
quart['Дата_new'] = quart['Дата'].apply(string)


In [87]:
def to_date(x):
    try:
        x = datetime.datetime.strptime(x, '%Y-%m-%d').date()
    except:
        x = datetime.datetime.strptime(x, '%d.%m.%Y').date()
    return x    

In [88]:
quart['Дата_new'] = quart['Дата_new'].apply(to_date)

In [89]:
quart.drop('Дата', axis=1, inplace=True)

In [90]:
quart = quart.rename(columns={'Дата_new': 'Дата'})

In [91]:
quart

,Клиент: телефон,Источник,Дата
0,79119846244,Одноклассники Звонки,2020-10-04
1,79217884353,Одноклассники Звонки,2020-11-07
2,79291050506,Тикток Звонки,2020-12-24
3,79376004981,Тикток Звонки,2020-11-30
4,79219838184,Яндекс Карты Звонки,2020-12-31
...,...,...,...
23356,8 (981) 795-4782,Яндекс Заявки,2020-11-04
23357,8 (911) 115-3190,Яндекс Заявки,2020-11-05
23358,8 (921) 944-5082,Яндекс Заявки,2020-11-05
23359,8 (921) 305-6348,Яндекс Заявки,2020-11-05


# re - телефоны

In [92]:
phones = quart['Клиент: телефон'].tolist()

pattern = r'\(\d{2}\)'
pattern2 = r'[а-яА-Яa-zA-Z]{1}'

for i, j in enumerate(phones):
    if len([x for x in j if x.isdigit()]) == 10:
        phones[i] = '7' + ''.join([x for x in j if x.isdigit()])    
    if len([x for x in j if x.isdigit()]) == 11:
        temp_arr = ''.join([x for x in j if x.isdigit()])
        if temp_arr[0] == '8':
            phones[i] = '7' + temp_arr[1:]
        elif temp_arr[0] == '7':    
            phones[i] = temp_arr

for i, j in enumerate(phones):
    if re.search(pattern, j):
        if '(61)' in j:
            phones[i] = phones[i].replace('(61)', ', 321-61-61, ')
        elif  '(39)' in j:   
            phones[i] = phones[i].replace('(39)', ', 305-73-39, ')
        elif '(73)' in j:
            phones[i] = phones[i].replace('(73)', '')
            phones[i] += ' , 558-73-09 '
        elif ('21') in j:
            phones[i] = phones[i].replace('(21)', '(921)')
    phones[i] = ''.join([x if x not in ',;!:.\\/+' else ' ' for x in j])        
    phones[i] = phones[i].lstrip(' ')
    phones[i] = phones[i].rstrip(' ')
    if re.search(pattern2, phones[i]):
        phones[i] = re.sub(pattern2, '', phones[i])
    phones[i] = ''.join([x for x in phones[i] if x not in '()-']) 
    
phones2 = phones.copy()

for i, j in enumerate(phones2):
    phones2[i] = j.split()
    
from functools import reduce
for i, j in enumerate(phones2):
    try:
        phones2[i].append(reduce(lambda x, y: x + y, [z for z in j if len(z) < 7]))
    except:
        continue
        
for i, j in enumerate(phones2):
    for x, y in enumerate(j):
        if x != len(j) - 1:
            if len(y) == 4 and len(j[x + 1]) == 7 and y[:2] in ('79', '89'):
                    phones2[i].append(y + j[x + 1])
            elif y in ('7', '8') and len(j[x + 1]) == 10:
                phones2[i].append(y + j[x + 1])
                
for i, j in enumerate(phones2):
    phones2[i] = [x for x in phones2[i] if len(x) >= 7]
    
for i, j in enumerate(phones2):
    for x, y in enumerate(j):
        if len(y) == 7:
            phones2[i][x] = '7812' + phones2[i][x]
        elif len(y) == 11 and y[0] == '8':
            phones2[i][x] = '7' + y[1:]
        elif len(y) == 10:
            phones2[i][x] = '7' + y    

In [93]:
quart['длина'] = quart['Клиент: телефон'].apply(len)

In [94]:
quart['Чистые телефоны'] = phones2

In [95]:
quart['Чистые телефоны'] = quart['Чистые телефоны'].apply(lambda x: ''.join(x) if isinstance(x, list) else x)

In [96]:
quart['длина чистые'] = quart['Чистые телефоны'].apply(len)

In [97]:
quart[quart['длина чистые'] != 11]


quart.drop([8975, 12675, 22492], inplace=True)



In [98]:
def clean_number(row):
    if row['длина чистые'] == 14:
        row['Чистые телефоны'] = '7' + row['Чистые телефоны'][4:]
    return row
        

In [99]:
quart = quart.apply(clean_number, axis=1)

In [101]:
quart['длина чистые'] = quart['Чистые телефоны'].apply(len)

In [107]:
pattern_7 = r'^7\d{10}$'
quart['соответствие'] = quart['Чистые телефоны'].apply(lambda x: bool(re.match(pattern_7, x)))

In [112]:
quart[~quart['соответствие']]

,Клиент: телефон,Источник,Дата,длина,Чистые телефоны,длина чистые,соответствие


In [111]:
quart.drop([1525], inplace=True)


In [114]:
quart.drop(['Клиент: телефон', 'длина', 'длина чистые', 'соответствие'], axis=1, inplace=True)

In [118]:
quart = quart.sort_values(by='Дата', ignore_index=True)
quart = quart.drop_duplicates(subset='Чистые телефоны', ignore_index=True)
quart

,Источник,Дата,Чистые телефоны
0,Вконтакте Звонки,2020-10-01,78137137416
1,Email рассылки Заявки,2020-10-01,79219022983
2,Вконтакте Звонки,2020-10-01,78137194058
3,Яндекс Звонки,2020-10-01,78124661490
4,Яндекс Звонки,2020-10-01,79217500051
...,...,...,...
23205,Email рассылки Заявки,2020-12-31,79516492981
23206,Гугл эдс Звонки,2020-12-31,79217454979
23207,Яндекс Карты Звонки,2020-12-31,79112618426
23208,Гугл эдс Звонки,2020-12-31,78126813400


# АУ

In [120]:
au_telefons = pd.read_excel('АУ.xlsx')
au_telefons = au_telefons.fillna('')

In [121]:
dom, mobil, work = au_telefons['Телефон домашний'].tolist(), au_telefons['Телефон мобильный'].tolist(), au_telefons['Телефон рабочий'].tolist()

In [122]:
pattern = r'\(\d{2}\)'
pattern2 = r'[а-яА-Яa-zA-Z]{1}'

for key in (dom, mobil, work):
    for i, j in enumerate(key):
        if len([x for x in j if x.isdigit()]) == 10:
            key[i] = '7' + ''.join([x for x in j if x.isdigit()])    
        if len([x for x in j if x.isdigit()]) == 11:
            temp_arr = ''.join([x for x in j if x.isdigit()])
            if temp_arr[0] == '8':
                key[i] = '7' + temp_arr[1:]
            elif temp_arr[0] == '7':    
                key[i] = temp_arr




for key in (dom, mobil, work):
    for i, j in enumerate(key):
        if type(j) == str:
            key[i] = ''.join([x if x not in ',;!:.\\/+' else ' ' for x in j])        
            key[i] = key[i].lstrip(' ')
            key[i] = key[i].rstrip(' ')
            if re.search(pattern2, key[i]):
                key[i] = re.sub(pattern2, '', key[i])
            key[i] = ''.join([x for x in key[i] if x not in '()-'])  

In [123]:
for key in (dom, mobil, work):
    for i, j in enumerate(key):
        if type(j) == str:
            if len(j) == 7:
                key[i] = '7812' + key[i]
            elif len(j) == 11 and j[0] == '8':
                key[i] = '7' + j[1:]
            elif len(j) == 10:
                key[i] = '7' + j

In [124]:
def concate_lists(*args) -> list:   # соединим все номера в одну строку чтобы проще было искать
    temp_lst = []
    for i in range(len(args[0])):
        temp_lst.append(' '.join([str(x[i]) for x in args]))
    return temp_lst
            
lst_search = concate_lists(dom, mobil, work)        

In [125]:
lst_search

['79817391105 79817391105 ',
 '79213423388 79213423388 ',
 '79219808081 79217778577 79219808081',
 ' 79216576413 ',
 '78122302362 79213184599 79219456920',
 '78127863432 79217444924 79217444924',
 ' 79217744340 ',
 ' 79217744340 ',
 ' 79217480201 79119266678',
 '79219695249 79219695249 79217807027',
 '79818938883 79818938883 ',
 ' 79219996617 78129521695',
 '79112292146 79112292146 79210933935',
 '79110958959 79112806263 79110958959',
 '79219005566 79219005566 ',
 ' 79219996617 78129521695',
 '79111823298 79111823298 ',
 '78122427036 79052107253 78122427036',
 ' 79110257113 ',
 '79626842212 79626842212 ',
 '79643399315 79643399315 ',
 '79112806263 79112806263 79112806263',
 ' 79270024925 79119332428',
 '79219457030 79219457030 ',
 ' 79811729004 ',
 '78122976439 79523722021 79046006439',
 ' 79214105131 79626860141',
 '79643399315 79643399315 ',
 ' 79045117915 79219374329',
 '79219470201 79219470201 ',
 '79112739349 79050137780 ',
 '79817391105 79817391105 ',
 '79214412122 79214412122 ',

# Сравнение

In [126]:
lst_with_coincidenc = []
for index, row in enumerate(lst_search):
    lst_with_coincidenc.append('')
    for i in quart['Чистые телефоны']:
        if str(i) in row:
            lst_with_coincidenc[index] = str(i)
            break
            
            
    

In [127]:
au_telefons['Совпадение'] = lst_with_coincidenc

In [169]:
au_telefons_final = au_telefons[au_telefons['Совпадение'] != '']

In [170]:
au_telefons_final = au_telefons_final.merge(quart, left_on='Совпадение', right_on='Чистые телефоны', how='left')

In [171]:
au_telefons_final = au_telefons_final.fillna('')

In [172]:
au_telefons_final['Старый/Новый'] = au_telefons_final['Дата приема'] == au_telefons_final['Минимальная дата приема Пациента (без удаленных)']
au_telefons_final['Старый/Новый'] = au_telefons_final['Старый/Новый'].replace({True:'Новый', False:'Старый'})
new_clients_lst = au_telefons_final[au_telefons_final['Старый/Новый'] == 'Новый']['Номер карты'].unique()
au_telefons_final.drop('Старый/Новый', axis=1, inplace=True)
au_telefons_final['Старый/Новый'] = au_telefons_final['Номер карты'].isin(new_clients_lst)
au_telefons_final['Старый/Новый'] = au_telefons_final['Старый/Новый'].replace({True:'Новый', False:'Старый'})


In [173]:
au_telefons_final

,Номер карты,Пациент,Дата приема,Минимальная дата приема Пациента (без удаленных),Специализация АУ_ОСН,Телефон мобильный,Телефон домашний,Телефон рабочий,Категория клиента ПЭО,Sum([Стоимость полная]),Sum([Количество]),Филиал,Совпадение,Источник,Дата,Чистые телефоны,Старый/Новый
0,173379,Устюжанин Владислав Максимович,2020-10-31,2010-06-08,нет,+7(911)280-62-63,+7(911)095-89-59,+7(911)095-89-59,АБ РЕН,2535.0,6,Б.С. 45,79110958959,Яндекс Карты Звонки,2020-12-20,79110958959,Старый
1,120452618,Кизилов Платон Кириллович,2020-12-01,2016-08-18,Аллерголог,+7(911)025-71-13,,,АБ РЕН,2520.0,5,Б.С. 45,79110257113,Яндекс Карты Звонки,2020-12-04,79110257113,Старый
2,100158911,Ковригин Федор Максимович,2020-12-28,2020-01-09,нет,+7(905)013-77-80,+7(911)273-93-49,,АБ РЕН,3710.0,4,Б.С. 45,79050137780,Яндекс Карты Звонки,2020-10-22,79050137780,Старый
3,173379,Устюжанин Владислав Максимович,2020-10-10,2010-06-08,нет,+7(911)280-62-63,+7(911)095-89-59,+7(911)095-89-59,АБ РЕН,750.0,3,Б.С. 45,79110958959,Яндекс Карты Звонки,2020-12-20,79110958959,Старый
4,120559412,Лабковская Ульяна Андреевна,2020-10-11,2018-07-13,Педиатр,+7(921)572-92-92,,+7(921)572-92-93,АБ РЕН,4025.0,2,Б.С. 45,79215729292,Яндекс Карты Звонки,2020-10-31,79215729292,Старый
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26808,990539453,Шейко Вера Викторовна,2020-12-08,2020-12-06,Терапевт,,,+7(914)787-78-78,-,0.0,1,БДВУ,79147877878,Яндекс Карты Звонки,2020-12-04,79147877878,Новый
26809,990539453,Шейко Вера Викторовна,2020-12-09,2020-12-06,Терапевт,,,+7(914)787-78-78,-,0.0,1,БДВУ,79147877878,Яндекс Карты Звонки,2020-12-04,79147877878,Новый
26810,990540119,Остонен Сергей Александрович,2020-12-20,2020-12-12,нет,+7(921)942-32-51,,+7(921)910-15-78,-,0.0,1,БДВУ,79219101578,Яндекс Карты Звонки,2020-12-10,79219101578,Старый
26811,990540959,Никитина Тамара Владимировна,2020-12-23,2020-12-18,Терапевт,,,+7(931)265-34-40,-,0.0,1,БДВУ,79312653440,Гугл эдс Звонки,2020-12-16,79312653440,Новый


In [174]:
au_telefons_final = au_telefons_final[au_telefons_final['Дата'] <= au_telefons_final['Дата приема']]

In [175]:
au_telefons_final['Уникальный клиент'] = (~au_telefons_final.duplicated(subset=['Номер карты'], keep='first')).astype(int)


C:\Users\AmanovRA\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [176]:
au_telefons_final.to_excel('Обработано.xlsx', index=False)